In [10]:
from bs4 import BeautifulSoup
import jsonlines

WINDOW_SIZE = 5  # تعداد پیام‌های اخیر قبل از پاسخ تو

def parse_html(path):
    with open(path, encoding="utf-8") as f:
        soup = BeautifulSoup(f, "html.parser")

    messages = []
    for msg in soup.find_all("div", class_="body"):
       
        if msg.find("div", class_="from_name"):
            name_tag = msg.find("div", class_="from_name")

        text_tag = msg.find("div", class_="text")
        if not text_tag:
            continue

        sender = name_tag.get_text(strip=True)
        text = text_tag.get_text(" ", strip=True)
        if not text: 
            continue

        role = "Me" if sender == "YesImBatMan" else "user"
        messages.append({"role": role, "content": text})
    return messages

def build_dataset(messages, window_size=WINDOW_SIZE):
    dataset = []
    for i in range(len(messages)):
        if messages[i]["role"] == "Me":
            # جمع کردن n پیام قبلش به عنوان ورودی
            context = messages[max(0, i-window_size):i]
            # اگر هیچ پیام کاربر نباشه بی‌فایده‌ست
            if not any(m["role"]=="user" for m in context):
                continue
            # ساخت ساختار دیتاست
            dataset.append({
                "messages": context + [messages[i]]
            })
    return dataset

if __name__ == "__main__":
    msgs = parse_html("messages.html")
    dataset = build_dataset(msgs, WINDOW_SIZE)

    with jsonlines.open("dataset.jsonl", "w") as writer:
        for d in dataset:
            writer.write(d)

    print("Saved", len(dataset), "samples in dataset.jsonl")


Saved 63 samples in dataset.jsonl
